In [1]:
import time
import copy
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
pd.set_option('display.max_columns', 100)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE


In [2]:
DATA_PATH = "train.csv"
#/kaggle/input/dry-beans-classification-iti-ai-pro-intake01/test.csv
data_path="/kaggle/input/dry-beans-classification-iti-ai-pro-intake01/train.csv"
data = pd.read_csv(data_path)


In [3]:
def more_features(df):
    df['ShapeFactor5'] = (4*df['Area'])/(df['MinorAxisLength']*df['MinorAxisLength'])
     #resources https://en.wikipedia.org/wiki/Eccentricity_(mathematics)
    df['Second_Eccentricity'] = df['Eccentricity']/np.sqrt(1-(df['Eccentricity']**2))  
    df['Third_Eccentricity'] = df['Eccentricity']/np.sqrt(2-(df['Eccentricity']**2))
    df['Angular_Eccentricity'] = np.arcsin(df['Eccentricity'])
    
    # resources https://en.wikipedia.org/wiki/Sphericity#Ellipsoidal_objects
    a = df['MajorAxisLength']
    b = df['MinorAxisLength']
    df['Sphericity'] = (2*(a*(b**2))**(1/3))/((a)+(b**2)/((a**2)-(b**2))*np.log((((a)+((a**2)-(b**2)))/(b**2))))
    
    # not implemented https://en.wikipedia.org/wiki/Roundness_(geology)#Measure_of_roundness
    ########################################################################################
    # https://en.wikipedia.org/wiki/Curvature 
    df['Curvature_R1'] = (b*b)/a
    df['Curvature_R2'] = (a*a)/b
    # https://en.wikipedia.org/wiki/Degree_of_curvature#Formula_from_radius
    df ['Curvature_Degree_1'] = 5729.58/df['Curvature_R1']
    df ['Curvature_Degree_2'] = 5729.58/df['Curvature_R2']
    
    return df

data = more_features(data)

In [4]:
cols_cat = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength',
       #'AspectRation',
        'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent',
       'Solidity', 'roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2',
       #'ShapeFactor3',
        'ShapeFactor4','ShapeFactor5',
       #'Second_Eccentricity', 
        #'Third_Eccentricity',
        #'Angular_Eccentricity',
       'Sphericity', 
        'Curvature_R1', 'Curvature_R2', 'Curvature_Degree_1',
       'Curvature_Degree_2' ,'y']

 
data_cat =  data[cols_cat]

# data=data.drop("ID",axis=1)
label_encoder = LabelEncoder()
robust_scaler = RobustScaler()

#robust_scaler.fit(data)
#data=
out_cat=label_encoder.fit_transform(data_cat.iloc[:, -1])
y_label_classes = label_encoder.classes_
n_samples = data_cat.shape[0]

X_cat=data_cat.drop(["y"],axis=1)
y_cat=out_cat

X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat, y_cat, test_size=0.20, random_state=42)

X_train_cat=X_train_cat.reset_index(drop=True)

X_test_cat=X_test_cat.reset_index(drop=True)

from catboost import CatBoostClassifier

clf_cat=CatBoostClassifier(silent=True,depth=5,iterations= 2000,random_state=42)

clf_cat = clf_cat.fit(X_train_cat,y_train_cat)


y_pred_train_cat = clf_cat.predict(X_train_cat)
y_pred_test_cat =  clf_cat.predict(X_test_cat)



clf_cat = clf_cat.fit(X_cat, y_cat)
y_pred_train_cat = clf_cat.predict(data_cat.drop(["y"],axis=1))
y_pred_test_cat =  clf_cat.predict(X_test_cat)

test_path="/kaggle/input/dry-beans-classification-iti-ai-pro-intake01/test.csv"
test_cat = pd.read_csv(test_path)
testcopy_cat=test_cat.copy()
test_cat=test_cat.drop("ID",axis=1)
test_cat = more_features(test_cat)
standard_scaler_cat = RobustScaler()

y_preds_cat=clf_cat.predict(test_cat)    

y_pred_cat=label_encoder.inverse_transform(y_preds_cat)


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [5]:
import time
import copy
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
torch.backends.cudnn.deterministic=True
from torch.utils.data import Dataset, DataLoader, TensorDataset
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.model_selection import train_test_split

DATA_PATH = "../input/dry-beans-classification-iti-ai-pro-intake01/train.csv"
data = pd.read_csv(DATA_PATH)
data.head()

data=data.drop("ID",axis=1)
X = torch.Tensor(np.array(data.iloc[:, :-1], dtype=np.float32))
label_encoder = LabelEncoder()
standard_scaler = RobustScaler()
out=label_encoder.fit_transform(data.iloc[:, -1])
y = torch.Tensor(out)
X = torch.Tensor(standard_scaler.fit_transform(X))
y_label_classes = label_encoder.classes_
n_samples = data.shape[0]

X.shape

class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(16,64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 7)

    def forward(self, x):
        output = self.fc1(x)
        output = self.relu1(output)
        output = self.fc2(output)
        output = self.relu2(output)
        output = self.fc3(output)
        return output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

train_data = TensorDataset(X_train, y_train.type(torch.LongTensor))
test_data = TensorDataset(X_test, y_test.type(torch.LongTensor))

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

batch_size = 64
learning_rate = 0.02
num_epoch = 300 #bs=32 epochs=300

network = MLP()
criterion = nn.CrossEntropyLoss()
 
# optimizer_ft = torch.optim.Adam(network.parameters(), lr=learning_rate)
optimizer_ft = torch.optim.SGD(network.parameters(), lr=0.02, momentum=0.9)
if train_on_gpu:
    network.cuda()

data = {"train": train_data, "val": test_data}
dataloaders = {x: torch.utils.data.DataLoader(data[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']}
dataset_sizes = {x: len(data[x]) for x in ['train', 'val']}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
               # inputs = inputs.to(device)
               # labels = labels.to(device)
                inputs,labels=inputs.cuda(),labels.cuda()
               # if train_on_gpu:
            #data, target = data.cuda(), target.cuda()
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    torch.save(best_model_wts, "savedmodel")
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


model_ft = train_model(network, criterion, optimizer_ft, num_epochs=num_epoch)

# preparing the test data

test = pd.read_csv("../input/dry-beans-classification-iti-ai-pro-intake01/test.csv")
testcopy=test.copy()
test=test.drop(["ID"],axis=1)
Xt = torch.Tensor(np.array(test, dtype=np.float32))
standard_scaler = RobustScaler()
Xt = torch.Tensor(standard_scaler.fit_transform(Xt))
n_samples =X.shape[0]

# I converted to the model to evaluation mode
model_ft.eval()

model_ft.to("cpu")

#inputs,labels=dataloaders["train"]
out=model_ft(Xt)     # the output is a tensor
_, preds = torch.max(out, 1)    #this takes the maximum probability for each output
y_preds=np.array(preds)      # converts from tensor to numpy array

y_pred_mlp=label_encoder.inverse_transform(y_preds)


CUDA is available!  Training on GPU ...
Epoch 0/299
----------
train Loss: 0.7898 Acc: 0.7392
val Loss: 0.3122 Acc: 0.8966

Epoch 1/299
----------
train Loss: 0.2582 Acc: 0.9113
val Loss: 0.2296 Acc: 0.9243

Epoch 2/299
----------
train Loss: 0.2313 Acc: 0.9181
val Loss: 0.2135 Acc: 0.9257

Epoch 3/299
----------
train Loss: 0.2222 Acc: 0.9204
val Loss: 0.2132 Acc: 0.9312

Epoch 4/299
----------
train Loss: 0.2164 Acc: 0.9205
val Loss: 0.2017 Acc: 0.9308

Epoch 5/299
----------
train Loss: 0.2151 Acc: 0.9248
val Loss: 0.1981 Acc: 0.9359

Epoch 6/299
----------
train Loss: 0.2120 Acc: 0.9230
val Loss: 0.2057 Acc: 0.9326

Epoch 7/299
----------
train Loss: 0.2092 Acc: 0.9228
val Loss: 0.2026 Acc: 0.9303

Epoch 8/299
----------
train Loss: 0.2073 Acc: 0.9248
val Loss: 0.2003 Acc: 0.9340

Epoch 9/299
----------
train Loss: 0.2037 Acc: 0.9255
val Loss: 0.1948 Acc: 0.9322

Epoch 10/299
----------
train Loss: 0.2012 Acc: 0.9257
val Loss: 0.1938 Acc: 0.9386

Epoch 11/299
----------
train Loss:

In [6]:
DATA_PATH = "train.csv"
data_path="/kaggle/input/dry-beans-classification-iti-ai-pro-intake01/train.csv"
data_lgbm = pd.read_csv(data_path)

label_encoder_lgbm = LabelEncoder()
robust_scaler_lgbm = RobustScaler()

out_lgbm=label_encoder_lgbm.fit_transform(data_lgbm.iloc[:, -1])
y_label_classes_lgbm = label_encoder_lgbm.classes_
n_samples_lgbm = data_lgbm.shape[0]

X_lgbm=data_lgbm.drop(["y"],axis=1)
X_lgbm=X_lgbm.drop("ID",axis=1)

y_lgbm=out_lgbm

X_train_lgbm, X_test_lgbm, y_train_lgbm, y_test_lgbm = train_test_split(X_lgbm, y_lgbm, test_size=0.20, random_state=42)

X_train_lgbm=X_train_lgbm.reset_index(drop=True)

X_test_lgbm=X_test_lgbm.reset_index(drop=True)

from sklearn.metrics import f1_score

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

from lightgbm import LGBMClassifier

params_lgbm={'num_leaves': 27,
 'max_depth': 1,
 'min_data_in_leaf': 64,
 'learning_rate': 0.15,
 'subsample': 0.9,
 'feature_fraction': 0.8,
 'n_estimators': 379,
 'colsample_bytree': 0.7
       }

clf_lgbm = LGBMClassifier(**params_lgbm)

clf_lgbm = clf_lgbm.fit(X_train,y_train)


y_pred_train_lgbm = clf_lgbm.predict(X_train)
y_pred_test_lgbm =  clf_lgbm.predict(X_test)

clf_lgbm = clf_lgbm.fit(X_lgbm, y_lgbm,)
y_pred_train_lgbm = clf_lgbm.predict(data_lgbm.drop(["y",'ID'],axis=1))
y_pred_test_lgbm =  clf_lgbm.predict(X_test_lgbm)

test_path="/kaggle/input/dry-beans-classification-iti-ai-pro-intake01/test.csv"
test_lgbm = pd.read_csv(test_path)
testcopy_lgbm=test_lgbm.copy()
test_lgbm=test_lgbm.drop("ID",axis=1)
standard_scaler_lgbm = RobustScaler()

y_preds_lgbm=clf_lgbm.predict(test_lgbm)    

y_pred_lgbm=label_encoder_lgbm.inverse_transform(y_preds_lgbm)



[LightGBM] [Warning] min_data_in_leaf is set=64, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=64
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.7 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=64, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=64
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.7 will be ignored. Current value: feature_fraction=0.8


In [7]:
from scipy import stats
new=pd.read_csv("../input/dry-beans-classification-iti-ai-pro-intake01/test.csv")

new['yLGBM'] = y_pred_lgbm
new['yCAT'] = y_pred_cat
new['yMLP'] = y_pred_mlp
new['y'] = stats.mode(new[['yLGBM','yCAT','yMLP']],axis=1).mode
new[['ID', 'y']].to_csv('new.csv', index=False)

In [8]:
new.head()

,ID,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,yLGBM,yCAT,yMLP,y
0,10834,57659,955.434,387.757607,196.625782,1.972059,0.861896,60188,270.949661,0.620790,0.957982,0.793735,0.698760,0.006725,0.000989,0.488266,0.962889,HOROZ,HOROZ,HOROZ,HOROZ
1,10835,28772,630.362,229.990785,159.609367,1.440960,0.719993,29127,191.399185,0.767458,0.987812,0.909913,0.832204,0.007994,0.002365,0.692563,0.997956,DERMASON,DERMASON,DERMASON,DERMASON
2,10836,54677,911.022,308.853903,226.398571,1.364204,0.680198,55858,263.850182,0.753013,0.978857,0.827860,0.854288,0.005649,0.001856,0.729808,0.995607,BARBUNYA,BARBUNYA,BARBUNYA,BARBUNYA
3,10837,24827,578.304,214.192699,147.788172,1.449322,0.723831,25121,177.794033,0.716508,0.988297,0.932871,0.830066,0.008627,0.002526,0.689009,0.998596,DERMASON,DERMASON,DERMASON,DERMASON
4,10838,141850,1432.847,524.923471,346.910920,1.513136,0.750492,143680,424.981211,0.804617,0.987263,0.868240,0.809606,0.003701,0.000981,0.655462,0.991803,BOMBAY,BOMBAY,BOMBAY,BOMBAY
